# Exercise 2

Required python packages

In [33]:
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
import numpy as np
import os
from logging import error, warning, info, basicConfig
from tqdm import tqdm
from numba import njit, jit, prange

In [38]:
np.random.seed(42)
basicConfig(level='INFO')
SAVE_PATH = 'temp_media/ex2'
ANIM_BACKEND = 'ffmpeg'
os.makedirs(SAVE_PATH, exist_ok=True)

Check for CUDA availability for GPU compute

In [40]:
from numba import cuda

CUDA = False

try:
    CUDA = cuda.detect()
    info("CUDA is available.")
except ImportError:
    info("CUDA is not available. Uses CPU for computations instead.")

INFO:root:CUDA is not available. Uses CPU for computations instead.
